In [1]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# from imblearn.over_sampling import SMOTER
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
data = pd.read_csv('cities_csv.csv')

In [3]:
data.head()

,Date,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,...,NC-Charlotte,NV-Las Vegas,NY-New York,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,National-US
0,1987-01-01,NaN,59.33,54.67,46.61,50.20,64.11,68.50,77.33,NaN,...,63.39,66.36,74.42,53.53,41.05,NaN,NaN,62.82,NaN,63.75
1,1987-02-01,NaN,59.65,54.89,46.87,49.96,64.77,68.76,77.93,NaN,...,63.94,67.03,75.43,53.50,41.28,NaN,NaN,63.39,NaN,64.15
2,1987-03-01,NaN,59.99,55.16,47.32,50.15,65.71,69.23,77.76,NaN,...,64.17,67.34,76.25,53.68,41.06,NaN,NaN,63.87,NaN,64.49
3,1987-04-01,NaN,60.81,55.85,47.69,50.55,66.40,69.20,77.56,NaN,...,64.81,67.88,77.34,53.75,40.96,NaN,NaN,64.57,NaN,64.99
4,1987-05-01,NaN,61.67,56.35,48.31,50.63,67.27,69.46,77.85,NaN,...,65.18,67.90,79.16,54.71,41.24,NaN,NaN,65.56,NaN,65.57


In [4]:
data.columns

Index(['Date', 'AZ-Phoenix', 'CA-Los Angeles', 'CA-San Diego',
       'CA-San Francisco', 'CO-Denver', 'DC-Washington', 'FL-Miami',
       'FL-Tampa', 'GA-Atlanta', 'IL-Chicago', 'MA-Boston', 'MI-Detroit',
       'MN-Minneapolis', 'NC-Charlotte', 'NV-Las Vegas', 'NY-New York',
       'OH-Cleveland', 'OR-Portland', 'TX-Dallas', 'WA-Seattle',
       'Composite-10', 'Composite-20', 'National-US'],
      dtype='object')

### Check for Missing value and try to get all the info related to it

In [5]:
data.isna().sum()

Date                  0
AZ-Phoenix           24
CA-Los Angeles        0
CA-San Diego          0
CA-San Francisco      0
CO-Denver             0
DC-Washington         0
FL-Miami              0
FL-Tampa              0
GA-Atlanta           48
IL-Chicago            0
MA-Boston             0
MI-Detroit           48
MN-Minneapolis       24
NC-Charlotte          0
NV-Las Vegas          0
NY-New York           0
OH-Cleveland          0
OR-Portland           0
TX-Dallas           156
WA-Seattle           36
Composite-10          0
Composite-20        156
National-US           0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              348 non-null    object 
 1   AZ-Phoenix        324 non-null    float64
 2   CA-Los Angeles    348 non-null    float64
 3   CA-San Diego      348 non-null    float64
 4   CA-San Francisco  348 non-null    float64
 5   CO-Denver         348 non-null    float64
 6   DC-Washington     348 non-null    float64
 7   FL-Miami          348 non-null    float64
 8   FL-Tampa          348 non-null    float64
 9   GA-Atlanta        300 non-null    float64
 10  IL-Chicago        348 non-null    float64
 11  MA-Boston         348 non-null    float64
 12  MI-Detroit        300 non-null    float64
 13  MN-Minneapolis    324 non-null    float64
 14  NC-Charlotte      348 non-null    float64
 15  NV-Las Vegas      348 non-null    float64
 16  NY-New York       348 non-null    float64
 1

#### First convert object into numerical

In [7]:

data['Date'] = pd.to_datetime(data['Date'])



data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day


In [8]:
data.head()

,Date,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,...,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,National-US,year,month,day
0,1987-01-01,NaN,59.33,54.67,46.61,50.20,64.11,68.50,77.33,NaN,...,53.53,41.05,NaN,NaN,62.82,NaN,63.75,1987,1,1
1,1987-02-01,NaN,59.65,54.89,46.87,49.96,64.77,68.76,77.93,NaN,...,53.50,41.28,NaN,NaN,63.39,NaN,64.15,1987,2,1
2,1987-03-01,NaN,59.99,55.16,47.32,50.15,65.71,69.23,77.76,NaN,...,53.68,41.06,NaN,NaN,63.87,NaN,64.49,1987,3,1
3,1987-04-01,NaN,60.81,55.85,47.69,50.55,66.40,69.20,77.56,NaN,...,53.75,40.96,NaN,NaN,64.57,NaN,64.99,1987,4,1
4,1987-05-01,NaN,61.67,56.35,48.31,50.63,67.27,69.46,77.85,NaN,...,54.71,41.24,NaN,NaN,65.56,NaN,65.57,1987,5,1


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              348 non-null    datetime64[ns]
 1   AZ-Phoenix        324 non-null    float64       
 2   CA-Los Angeles    348 non-null    float64       
 3   CA-San Diego      348 non-null    float64       
 4   CA-San Francisco  348 non-null    float64       
 5   CO-Denver         348 non-null    float64       
 6   DC-Washington     348 non-null    float64       
 7   FL-Miami          348 non-null    float64       
 8   FL-Tampa          348 non-null    float64       
 9   GA-Atlanta        300 non-null    float64       
 10  IL-Chicago        348 non-null    float64       
 11  MA-Boston         348 non-null    float64       
 12  MI-Detroit        300 non-null    float64       
 13  MN-Minneapolis    324 non-null    float64       
 14  NC-Charlotte      348 non-

In [10]:
data.isna().sum()

Date                  0
AZ-Phoenix           24
CA-Los Angeles        0
CA-San Diego          0
CA-San Francisco      0
CO-Denver             0
DC-Washington         0
FL-Miami              0
FL-Tampa              0
GA-Atlanta           48
IL-Chicago            0
MA-Boston             0
MI-Detroit           48
MN-Minneapolis       24
NC-Charlotte          0
NV-Las Vegas          0
NY-New York           0
OH-Cleveland          0
OR-Portland           0
TX-Dallas           156
WA-Seattle           36
Composite-10          0
Composite-20        156
National-US           0
year                  0
month                 0
day                   0
dtype: int64

In [11]:
data['AZ-Phoenix'].fillna(data['AZ-Phoenix'].median(), inplace=True)
data['GA-Atlanta'].fillna(data['GA-Atlanta'].median(), inplace=True)
data['MI-Detroit'].fillna(data['MI-Detroit'].median(), inplace=True)
data['MN-Minneapolis'].fillna(data['MN-Minneapolis'].median(), inplace=True)
data['TX-Dallas'].fillna(data['TX-Dallas'].median(), inplace=True)
data['WA-Seattle'].fillna(data['WA-Seattle'].median(), inplace=True)
data['Composite-20'].fillna(data['Composite-20'].median(), inplace=True)

In [12]:
data.isna().sum()

Date                0
AZ-Phoenix          0
CA-Los Angeles      0
CA-San Diego        0
CA-San Francisco    0
CO-Denver           0
DC-Washington       0
FL-Miami            0
FL-Tampa            0
GA-Atlanta          0
IL-Chicago          0
MA-Boston           0
MI-Detroit          0
MN-Minneapolis      0
NC-Charlotte        0
NV-Las Vegas        0
NY-New York         0
OH-Cleveland        0
OR-Portland         0
TX-Dallas           0
WA-Seattle          0
Composite-10        0
Composite-20        0
National-US         0
year                0
month               0
day                 0
dtype: int64

### As data is cleaned now lets do Normalization 

In [13]:
# # Standard Scaling
# # Select numerical columns only
# numeric_columns = data.select_dtypes(include=['float64', 'int64'])

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Standardize the numerical columns
# data[numeric_columns.columns] = scaler.fit_transform(numeric_columns)


In [14]:
data.head()

,Date,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,...,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,National-US,year,month,day
0,1987-01-01,105.905,59.33,54.67,46.61,50.20,64.11,68.50,77.33,105.255,...,53.53,41.05,118.43,116.03,62.82,148.685,63.75,1987,1,1
1,1987-02-01,105.905,59.65,54.89,46.87,49.96,64.77,68.76,77.93,105.255,...,53.50,41.28,118.43,116.03,63.39,148.685,64.15,1987,2,1
2,1987-03-01,105.905,59.99,55.16,47.32,50.15,65.71,69.23,77.76,105.255,...,53.68,41.06,118.43,116.03,63.87,148.685,64.49,1987,3,1
3,1987-04-01,105.905,60.81,55.85,47.69,50.55,66.40,69.20,77.56,105.255,...,53.75,40.96,118.43,116.03,64.57,148.685,64.99,1987,4,1
4,1987-05-01,105.905,61.67,56.35,48.31,50.63,67.27,69.46,77.85,105.255,...,54.71,41.24,118.43,116.03,65.56,148.685,65.57,1987,5,1


In [15]:
# # Min-max
# numeric_columns = data.select_dtypes(include=['float64', 'int64'])

# # Initialize the MinMaxScaler
# min_scaler = MinMaxScaler()

# # Min-Max scaling on numerical columns
# data[numeric_columns.columns] = min_scaler.fit_transform(numeric_columns)


In [16]:
data.head()

,Date,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,...,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,National-US,year,month,day
0,1987-01-01,105.905,59.33,54.67,46.61,50.20,64.11,68.50,77.33,105.255,...,53.53,41.05,118.43,116.03,62.82,148.685,63.75,1987,1,1
1,1987-02-01,105.905,59.65,54.89,46.87,49.96,64.77,68.76,77.93,105.255,...,53.50,41.28,118.43,116.03,63.39,148.685,64.15,1987,2,1
2,1987-03-01,105.905,59.99,55.16,47.32,50.15,65.71,69.23,77.76,105.255,...,53.68,41.06,118.43,116.03,63.87,148.685,64.49,1987,3,1
3,1987-04-01,105.905,60.81,55.85,47.69,50.55,66.40,69.20,77.56,105.255,...,53.75,40.96,118.43,116.03,64.57,148.685,64.99,1987,4,1
4,1987-05-01,105.905,61.67,56.35,48.31,50.63,67.27,69.46,77.85,105.255,...,54.71,41.24,118.43,116.03,65.56,148.685,65.57,1987,5,1


In [17]:
data.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['Date', 'AZ-Phoenix', 'CA-Los Angeles', 'CA-San Diego',
       'CA-San Francisco', 'CO-Denver', 'DC-Washington', 'FL-Miami',
       'FL-Tampa', 'GA-Atlanta', 'IL-Chicago', 'MA-Boston', 'MI-Detroit',
       'MN-Minneapolis', 'NC-Charlotte', 'NV-Las Vegas', 'NY-New York',
       'OH-Cleveland', 'OR-Portland', 'TX-Dallas', 'WA-Seattle',
       'Composite-10', 'Composite-20', 'National-US', 'year', 'month', 'day'],
      dtype='object')>

In [18]:
x = data.drop(['National-US', 'Date'], axis=1)


y = data['National-US']

In [19]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AZ-Phoenix        348 non-null    float64
 1   CA-Los Angeles    348 non-null    float64
 2   CA-San Diego      348 non-null    float64
 3   CA-San Francisco  348 non-null    float64
 4   CO-Denver         348 non-null    float64
 5   DC-Washington     348 non-null    float64
 6   FL-Miami          348 non-null    float64
 7   FL-Tampa          348 non-null    float64
 8   GA-Atlanta        348 non-null    float64
 9   IL-Chicago        348 non-null    float64
 10  MA-Boston         348 non-null    float64
 11  MI-Detroit        348 non-null    float64
 12  MN-Minneapolis    348 non-null    float64
 13  NC-Charlotte      348 non-null    float64
 14  NV-Las Vegas      348 non-null    float64
 15  NY-New York       348 non-null    float64
 16  OH-Cleveland      348 non-null    float64
 1

In [20]:
x.head()

,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,IL-Chicago,...,NY-New York,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,year,month,day
0,105.905,59.33,54.67,46.61,50.20,64.11,68.50,77.33,105.255,53.55,...,74.42,53.53,41.05,118.43,116.03,62.82,148.685,1987,1,1
1,105.905,59.65,54.89,46.87,49.96,64.77,68.76,77.93,105.255,54.64,...,75.43,53.50,41.28,118.43,116.03,63.39,148.685,1987,2,1
2,105.905,59.99,55.16,47.32,50.15,65.71,69.23,77.76,105.255,54.80,...,76.25,53.68,41.06,118.43,116.03,63.87,148.685,1987,3,1
3,105.905,60.81,55.85,47.69,50.55,66.40,69.20,77.56,105.255,54.88,...,77.34,53.75,40.96,118.43,116.03,64.57,148.685,1987,4,1
4,105.905,61.67,56.35,48.31,50.63,67.27,69.46,77.85,105.255,55.43,...,79.16,54.71,41.24,118.43,116.03,65.56,148.685,1987,5,1


In [21]:
x.shape,y.shape

((348, 25), (348,))

In [22]:
len(x),len(y)

(348, 348)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x.head()

,AZ-Phoenix,CA-Los Angeles,CA-San Diego,CA-San Francisco,CO-Denver,DC-Washington,FL-Miami,FL-Tampa,GA-Atlanta,IL-Chicago,...,NY-New York,OH-Cleveland,OR-Portland,TX-Dallas,WA-Seattle,Composite-10,Composite-20,year,month,day
0,105.905,59.33,54.67,46.61,50.20,64.11,68.50,77.33,105.255,53.55,...,74.42,53.53,41.05,118.43,116.03,62.82,148.685,1987,1,1
1,105.905,59.65,54.89,46.87,49.96,64.77,68.76,77.93,105.255,54.64,...,75.43,53.50,41.28,118.43,116.03,63.39,148.685,1987,2,1
2,105.905,59.99,55.16,47.32,50.15,65.71,69.23,77.76,105.255,54.80,...,76.25,53.68,41.06,118.43,116.03,63.87,148.685,1987,3,1
3,105.905,60.81,55.85,47.69,50.55,66.40,69.20,77.56,105.255,54.88,...,77.34,53.75,40.96,118.43,116.03,64.57,148.685,1987,4,1
4,105.905,61.67,56.35,48.31,50.63,67.27,69.46,77.85,105.255,55.43,...,79.16,54.71,41.24,118.43,116.03,65.56,148.685,1987,5,1


In [24]:
def regression_models_evaluation(X_train, X_test, y_train, y_test):
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'LightGBM': LGBMRegressor(),
        'XGBoost': XGBRegressor()
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Model: {name}")
        print(f"Mean Squared Error: {mse:.4f}")
        print(f"Mean Absolute Error: {mae:.4f}")
        print(f"R^2 Score: {r2:.4f}")
        print("------------------------------------")


In [25]:
regression_models_evaluation(X_train, X_test, y_train, y_test)


Model: Linear Regression
Mean Squared Error: 0.5565
Mean Absolute Error: 0.5926
R^2 Score: 0.9996
------------------------------------
Model: Ridge Regression
Mean Squared Error: 0.5532
Mean Absolute Error: 0.5893
R^2 Score: 0.9996
------------------------------------
Model: Lasso Regression
Mean Squared Error: 1.1302
Mean Absolute Error: 0.7659
R^2 Score: 0.9992
------------------------------------
Model: Decision Tree
Mean Squared Error: 1.2235
Mean Absolute Error: 0.7054
R^2 Score: 0.9991
------------------------------------


c:\Users\Amit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.221e+02, tolerance: 4.255e+01
  model = cd_fast.enet_coordinate_descent(


Model: Random Forest
Mean Squared Error: 0.6669
Mean Absolute Error: 0.4383
R^2 Score: 0.9995
------------------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1973
[LightGBM] [Info] Number of data points in the train set: 278, number of used features: 24
[LightGBM] [Info] Start training from score 119.750827
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

### We will take Random Forest as our model

In [32]:
X_test['Date'] = pd.to_datetime(X_test[['year', 'month', 'day']].astype(str).apply('-'.join, axis=1))
y_test_df = pd.DataFrame({'Date': X_test['Date'], 'National': y_test})

y_test_df.to_csv('output.csv', index=False)


In [31]:
# X_test['Date'][:10]

255   2008-04-01
114   1996-07-01
314   2013-03-01
268   2009-05-01
167   2000-12-01
124   1997-05-01
203   2003-12-01
224   2005-09-01
284   2010-09-01
153   1999-10-01
Name: Date, dtype: datetime64[ns]